# Stochastic Differential Equations (SDEs)

- [Wikipedia: Stochastic Differential Equations](https://en.wikipedia.org/wiki/Stochastic_differential_equation)
- [SDE example](https://diffeq.sciml.ai/stable/tutorials/sde_example/) in `DifferentialEquations.jl`

## Scalar SDEs

Solving the equation: $du=f(u,p,t)dt + g(u,p,t)dW$

- $f(u,p,t)$ is the ordinary differential equations (ODEs) part
- $g(u,p,t)$ is the stochastic part, paired with a Brownian motion term $dW$.

If $f(u,p,t) = \alpha u$ and $g(u,p,t) = \beta u$, the analytical solution is $u(t, W_t) = u_0 exp((\alpha - \frac{\beta^2}{2})t + \beta W_t)$

In [ ]:
using DifferentialEquations
using Plots

f(u, p, t) = p.α * u
g(u, p, t) = p.β * u

p = (α=1, β=1)
u₀ = 1/2
dt = 1//2^(4)
tspan = (0.0,1.0)
prob = SDEProblem(f, g, u₀, (0.0,1.0), p)

In [ ]:
# use the classic Euler-Maruyama algorithm
sol = solve(prob,EM(), dt=dt)
plot(sol)

In [ ]:
# Compare to the analytical soolution
f_analytic(u₀, p, t, W) = u₀ * exp((p.α - (p.β^2) / 2) * t + p.β * W)
# Combine model and analytic functions
ff = SDEFunction(f, g, analytic=f_analytic)
prob = SDEProblem(ff, g, u₀, (0.0,1.0), p)
sol = solve(prob, EM(), dt=dt)
plot(sol, plot_analytic=true)

In [ ]:
# choose a higher-order solver for a more accurate result
sol = solve(prob, SRIW1(), dt=dt, adaptive=false)
plot(sol, plot_analytic=true)

In [ ]:
# The solver can find dt itself
sol = solve(prob, SRIW1())
plot(sol, plot_analytic=true)

## Systems of SDEs with Diagonal Noise

In [ ]:
function lorenz!(du,u,p,t)
    du[1] = 10.0(u[2]-u[1])
    du[2] = u[1]*(28.0-u[3]) - u[2]
    du[3] = u[1]*u[2] - (8/3)*u[3]
end
  
function σ_lorenz!(du,u,p,t)
    du[1] = 3.0
    du[2] = 3.0
    du[3] = 3.0
end
  
prob_sde_lorenz = SDEProblem(lorenz!,σ_lorenz!,[1.0,0.0,0.0],(0.0,20.0))
sol = solve(prob_sde_lorenz)
plot(sol,vars=(1,2,3))

## Systems of SDEs with Scalar Noise

The same noise process (`W`) is applied to all SDEs.

In [ ]:
f(du,u,p,t) = (du .= u)
g(du,u,p,t) = (du .= u)
u0 = rand(4,2)

W = WienerProcess(0.0,0.0,0.0)
prob = SDEProblem(f,g,u0,(0.0,1.0),noise=W)
sol = solve(prob,SRIW1())
plot(sol)

## Random ODEs

Random ODEs (RODEs) is a intermediate between ODE and SDE. $du = f(u,p,t,W) dt$ where $W(t)$ is a Wiener process (Gaussian process).

Use `RODEProblem(f,u0,tspan)` to construct a RODE problem.

### Scalar RODEs

$du = f(u,p,t,W) = 2usin(W)$

In [ ]:
using DifferentialEquations
using Plots

function f(u,p,t,W)
  2u * sin(W)
end

u0 = 1.00
tspan = (0.0, 5.0)
prob = RODEProblem(f, u0, tspan)
sol = solve(prob, RandomEM(), dt=1/100)

plot(sol)

### Systems of RODEs

Using the in-place form for the right-jand-side (RHS).

In [ ]:
using DifferentialEquations
using Plots

function f(du, u, p, t, W)
  du[1] = 2u[1] * sin(W[1] - W[2])
  du[2] = -2u[2] * cos(W[1] + W[2])
end

u0 = [1.00; 1.00]
tspan = (0.0, 5.0)

prob = RODEProblem(f,u0,tspan)
sol = solve(prob, RandomEM(), dt=1/100)

plot(sol)